--- 
Project for the course in Microeconometrics | Summer 2021, M.Sc. Economics, Bonn University | [Aysu Avcı](https://github.com/aysuavci)

# Replication of Zimmermann (2020) <a class="tocSkip">   
---

This notebook contains my replication of the results from the following paper:

> Zimmermann, F. (2020). The Dynamics of Motivated Beliefs. American Economic Review, 110(2), 337–363.

##### Downloading and viewing this notebook:

* The best way to view this notebook is by downloading it and the repository it is located in from [GitHub](https://github.com/HumanCapitalAnalysis/template-course-project). Other viewing options like _MyBinder_ or _NBViewer_ may have issues with displaying images or coloring of certain parts (missing images can be viewed in the folder [files](https://github.com/HumanCapitalAnalysis/template-course-project/tree/master/files) on GitHub).


* The original paper, as well as the data and code provided by the authors can be accessed [here](https://www.aeaweb.org/articles?id=10.1257/aer.20180728).

##### Information about replication and individual contributions:

* For the replication, I try to remain true to the original structure of the paper so readers can easily follow along and compare. All tables and figures are named and labeled as they appear in Lindo et al. (2010).


* The tables in my replication appear transposed compared to the original tables to suit my workflow in Python.


* For transparency, all sections in the replication that constitute independent contributions by me and are not part of results presented (or include deviations from the methods used) in the paper are marked as _extensions_.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Introduction" data-toc-modified-id="1.-Introduction-1">1. Introduction</a></span></li><li><span><a href="#2.-Theoretical-Background" data-toc-modified-id="2.-Theoretical-Background-2">2. Theoretical Background</a></span></li><li><span><a href="#3.-Identification" data-toc-modified-id="3.-Identification-3">3. Identification</a></span></li><li><span><a href="#4.-Empirical-Strategy" data-toc-modified-id="4.-Empirical-Strategy-4">4. Empirical Strategy</a></span></li><li><span><a href="#5.-Replication-of-Lindo-et-al.-(2010)" data-toc-modified-id="5.-Replication-of-Lindo-et-al.-(2010)-5">5. Replication of Lindo et al. (2010)</a></span><ul class="toc-item"><li><span><a href="#5.1.-Data-&amp;-Descriptive-Statistics" data-toc-modified-id="5.1.-Data-&amp;-Descriptive-Statistics-5.1">5.1. Data &amp; Descriptive Statistics</a></span></li><li><span><a href="#5.2.-Results" data-toc-modified-id="5.2.-Results-5.2">5.2. Results</a></span><ul class="toc-item"><li><span><a href="#5.2.1.-Tests-of-the-Validity-of-the-RD-Approach" data-toc-modified-id="5.2.1.-Tests-of-the-Validity-of-the-RD-Approach-5.2.1">5.2.1. Tests of the Validity of the RD Approach</a></span></li><li><span><a href="#i.--Extension:-Visual-Validity-Check" data-toc-modified-id="i.--Extension:-Visual-Validity-Check-5.2.2">i.  Extension: Visual Validity Check</a></span></li><li><span><a href="#ii.-Advanced-Validity-Check" data-toc-modified-id="ii.-Advanced-Validity-Check-5.2.3">ii. Advanced Validity Check</a></span></li><li><span><a href="#5.2.2.-First-Year-GPAs-and-Academic-Probation" data-toc-modified-id="5.2.2.-First-Year-GPAs-and-Academic-Probation-5.2.4">5.2.2. First Year GPAs and Academic Probation</a></span></li><li><span><a href="#5.2.3.-The-Immediate-Response-to-Academic-Probation" data-toc-modified-id="5.2.3.-The-Immediate-Response-to-Academic-Probation-5.2.5">5.2.3. The Immediate Response to Academic Probation</a></span></li><li><span><a href="#5.2.4.-The-Impact-onSubsequent-Performance" data-toc-modified-id="5.2.4.-The-Impact-onSubsequent-Performance-5.2.6">5.2.4. The Impact onSubsequent Performance</a></span></li><li><span><a href="#i.-Main-Results-for-Impact-on-GPA-&amp;-Probability-of-Placing-Above-Cutoff-in-the-Next-Term" data-toc-modified-id="i.-Main-Results-for-Impact-on-GPA-&amp;-Probability-of-Placing-Above-Cutoff-in-the-Next-Term-5.2.7">i. Main Results for Impact on GPA &amp; Probability of Placing Above Cutoff in the Next Term</a></span></li><li><span><a href="#ii.-Formal-Bound-Analysis-on-Subsequent-GPA-(partial-extension)" data-toc-modified-id="ii.-Formal-Bound-Analysis-on-Subsequent-GPA-(partial-extension)-5.2.8">ii. Formal Bound Analysis on Subsequent GPA (partial extension)</a></span></li><li><span><a href="#5.2.5.-The-Impacts-on-Graduation" data-toc-modified-id="5.2.5.-The-Impacts-on-Graduation-5.2.9">5.2.5. The Impacts on Graduation</a></span></li></ul></li></ul></li><li><span><a href="#6.-Extension:-Robustness-Checks" data-toc-modified-id="6.-Extension:-Robustness-Checks-6">6. Extension: Robustness Checks</a></span><ul class="toc-item"><li><span><a href="#6.1.--A-Closer-Look-at-Students'-Subsequent-Performance." data-toc-modified-id="6.1.--A-Closer-Look-at-Students'-Subsequent-Performance.-6.1">6.1.  A Closer Look at Students' Subsequent Performance.</a></span><ul class="toc-item"><li><span><a href="#6.1.1.-Subsequent-Performance-and-Total-Credits-in-Year-2" data-toc-modified-id="6.1.1.-Subsequent-Performance-and-Total-Credits-in-Year-2-6.1.1">6.1.1. Subsequent Performance and Total Credits in Year 2</a></span></li><li><span><a href="#6.1.2.-Subsequent-Cumulative-Grade-Point-Average-(CGPA)" data-toc-modified-id="6.1.2.-Subsequent-Cumulative-Grade-Point-Average-(CGPA)-6.1.2">6.1.2. Subsequent Cumulative Grade Point Average (CGPA)</a></span></li></ul></li><li><span><a href="#6.2.-Bandwidth-Sensitivity" data-toc-modified-id="6.2.-Bandwidth-Sensitivity-6.2">6.2. Bandwidth Sensitivity</a></span></li></ul></li><li><span><a href="#7.-Conclusion" data-toc-modified-id="7.-Conclusion-7">7. Conclusion</a></span></li><li><span><a href="#8.-References" data-toc-modified-id="8.-References-8">8. References</a></span></li></ul></div>

In [22]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api
import matplotlib as plt
from IPython.display import HTML

In [23]:
from auxiliary.example_project_auxiliary_predictions import *
from auxiliary.example_project_auxiliary_plots import *
from auxiliary.example_project_auxiliary_tables import *

---
# Introduction 
---

---
# 2. Theoretical Background
---


---
# 3. Identification
--- 


---
# 4. Empirical Strategy
---


---
# 5. Replication of Zimmermann (2020)
---

## 5.1. Data & Descriptive Statistics
Data Setup
* creating a variable named 'treatgroup' to group different treatments
* <span style="color:red">panel data adjustment - rank??
* generating bayesian predictions

In [24]:
df = pd.read_stata('data/data.dta')

In [25]:
#For better viewing
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 7)
pd.set_option("display.max.rows", 30)

In [26]:
#Creating treatgroup column
tg_conditions = [
    (df['treatment'] == 'belief_announcement'),
    (df['treatment'] == 'confidence_1monthlater'),
    (df['treatment'] == 'confidence_direct_15minuteslater'),
    (df['treatment'] == 'confidence_direct_immediate'),
    (df['treatment'] == 'memory'),
    (df['treatment'] == 'memory_high'),
    (df['treatment'] == 'nofeedback'),
    (df['treatment'] == 'tournament_announcement'),
]
tg_values = [1, 2, 3, 4, 5, 6, 7, 8]
df['treatgroup'] = np.select(tg_conditions, tg_values)

In [27]:
#Generating bayesian predictions
df["post_1"] = df["prior_1"]/100

df.loc[df['pos_comparisons'] == 0, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 1, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 2, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 3, 'post_1'] = df["prior_1"]/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_2"] = df["prior_2"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_2'] = (df["prior_2"]*(1/9)*(1/9)*(1/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_2'] = (df["prior_2"]*(24/9)*(1/9)*(1/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_2'] = (df["prior_2"]*(24/9)*(8/9)*(1/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_2'] = (df["prior_2"]*(8/9)*(8/9)*(8/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_3"] = df["prior_3"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_3'] = (df["prior_3"]*(2/9)*(2/9)*(2/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_3'] = (df["prior_3"]*(21/9)*(2/9)*(2/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_3'] = (df["prior_3"]*(21/9)*(7/9)*(2/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_3'] = (df["prior_3"]*(7/9)*(7/9)*(7/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_4"] = df["prior_4"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_4'] = (df["prior_4"]*(3/9)*(3/9)*(3/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_4'] = (df["prior_4"]*(18/9)*(3/9)*(3/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_4'] = (df["prior_4"]*(18/9)*(6/9)*(3/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_4'] = (df["prior_4"]*(6/9)*(6/9)*(6/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_5"] = df["prior_5"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_5'] = (df["prior_5"]*(4/9)*(4/9)*(4/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_5'] = (df["prior_5"]*(15/9)*(4/9)*(4/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_5'] = (df["prior_5"]*(15/9)*(5/9)*(4/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_5'] = (df["prior_5"]*(5/9)*(5/9)*(5/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_6"] = df["prior_6"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_6'] = (df["prior_6"]*(5/9)*(5/9)*(5/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_6'] = (df["prior_6"]*(12/9)*(5/9)*(5/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_6'] = (df["prior_6"]*(12/9)*(4/9)*(5/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_6'] = (df["prior_6"]*(4/9)*(4/9)*(4/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_7"] = df["prior_7"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_7'] = (df["prior_7"]*(6/9)*(6/9)*(6/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_7'] = (df["prior_7"]*(9/9)*(6/9)*(6/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_7'] = (df["prior_7"]*(9/9)*(3/9)*(6/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_7'] = (df["prior_7"]*(3/9)*(3/9)*(3/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_8"] = df["prior_8"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_8'] = (df["prior_8"]*(7/9)*(7/9)*(7/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_8'] = (df["prior_8"]*(6/9)*(7/9)*(7/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_8'] = (df["prior_8"]*(6/9)*(2/9)*(7/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_8'] = (df["prior_8"]*(2/9)*(2/9)*(2/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_9"] = df["prior_9"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_9'] = (df["prior_9"]*(8/9)*(8/9)*(8/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_9'] = (df["prior_9"]*(3/9)*(8/9)*(8/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_9'] = (df["prior_9"]*(3/9)*(1/9)*(8/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_9'] = (df["prior_9"]*(1/9)*(1/9)*(1/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_10"] = df["prior_10"]/100

df.loc[df['pos_comparisons'] == 0, 'post_10'] = df["prior_10"]/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9) + df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_10'] = 0
df.loc[df['pos_comparisons'] == 2, 'post_10'] = 0
df.loc[df['pos_comparisons'] == 3, 'post_10'] = 0

In [28]:
#To check the new variables
df

,treatment,rscore,performance1,performance2,performance3,performance4,performance5,performance6,performance7,performance8,performance9,wspt,donation,prior_median,prior_1,prior_2,prior_3,prior_4,prior_5,prior_6,prior_7,prior_8,prior_9,prior_10,rank,pos_comparisons,neg_comparisons,repeat_neg_comparisons,repeat_pos_comparisons,risk_investment,recall_neg_comparisons,recall_pos_comparisons,big5_1,big5_2,big5_3,big5_4,big5_5,big5_6,big5_7,big5_8,big5_9,big5_10,big5_11,big5_12,big5_13,big5_14,big5_15,gender,field_of_study,student,posterior_median,recallfeedback,qualtrics,recall_part_IQtest,recall_part_feedback,recall_parts_nonIQ,pricelist1,pricelist2,pricelist3,pricelist4,pricelist5,pricelist6,pricelist7,pricelist8,pricelist9,pricelist10,pricelist11,pricelist12,pricelist13,pricelist14,pricelist15,treatgroup,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10
0,belief_announcement,4,1,3,4,4,5,5,5,6,8,6,1,53,2,4,9,14,24,16,13,9,7,2,7,2.0,1.0,1.0,2.0,80,1.0,2.0,1,2,4,3,2,4,3,4,2,5,3,4,3,4,4,0,Geographie,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0348774,0.1201635,0.2059946,0.3269755,0.1743869,0.0956403,0.0343324,0.0076294,0.0000000
1,belief_announcement,6,3,4,5,5,6,6,7,8,9,7,5,70,10,10,10,20,20,10,5,5,5,5,4,2.0,1.0,1.0,2.0,150,1.0,2.0,2,2,6,2,5,2,7,2,2,7,3,6,2,2,3,1,psychologie,1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0910384,0.1394026,0.3072546,0.2844950,0.1137980,0.0384068,0.0199147,0.0056899,0.0000000
2,belief_announcement,8,4,6,7,7,7,8,8,9,10,5,3,85,1,4,30,30,20,9,2,2,1,1,3,3.0,0.0,0.0,2.0,80,0.0,2.0,3,5,6,3,2,3,4,4,2,2,3,2,2,3,5,1,Medienwissenschaft,1,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.032123,0.0902441,0.4534238,0.2855380,0.1101613,0.0253812,0.0023795,0.0007050,0.0000441,0.0000000
3,belief_announcement,5,4,6,7,7,7,8,8,9,10,5,0,80,3,6,6,25,40,5,5,4,3,3,9,0.0,3.0,3.0,0.0,150,3.0,0.0,1,2,7,3,2,2,6,4,2,4,1,4,1,2,3,0,Lebensmittelchemie,1,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0005947,0.0047577,0.0669045,0.2537417,0.0619487,0.1070473,0.1359897,0.1522450,0.2167707
4,belief_announcement,5,1,3,4,4,5,5,5,6,8,3,0,25,0,0,0,0,25,5,5,20,20,25,4,3.0,0.0,0.0,3.0,100,0.0,3.0,4,4,7,3,2,1,7,2,6,3,2,4,1,1,4,1,Geographie,1,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0000000,0.0000000,0.0000000,0.8311170,0.0851064,0.0359043,0.0425532,0.0053191,0.0000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,tournament_announcement,7,3,4,5,5,6,6,7,8,9,5,0,80,5,10,40,15,10,4,4,4,4,4,3,2.0,1.0,1.0,2.0,0,1.0,2.0,2,2,6,2,5,4,6,2,4,5,3,5,2,3,3,0,Wirtschaftsinformatik,1,60.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0814249,0.4987277,0.2061069,0.1272265,0.0407125,0.0274809,0.0142494,0.0040712,0.0000000
738,tournament_announcement,7,4,6,7,7,7,8,8,9,10,4,4,75,10,10,30,15,10,10,5,5,5,0,8,0.0,3.0,3.0,0.0,200,3.0,0.0,1,1,2,2,2,3,6,1,1,3,1,7,2,1,3,1,Lehramt Englisch/Biologie Gym/Ge,1,50.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8,0.000000,0.0012658,0.0303797,0.0512658,0.0810127,0.1582278,0.1367089,0.2170886,0.3240506,0.0000000
739,tournament_announcement,7,4,6,7,7,7,8,8,9,10,5,0,70,2,8,20,30,10,12,8,6,3,1,6,0.0,3.0,3.0,0.0,0,3.0,0.0,3,2,5,3,4,3,5,4,3,3,3,2,3,4,4,0,Volkswirtschaftslehre,1,50.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0008725,0.0174501,0.0883411,0.0698004,0.1635947,0.1884611,0.2244520,0.1675210,0.0795070
740,tournament_announcement,5,4,6,7

In [29]:
#Generate posterior_median_bayes

df["posterior_median_bayes"] = (df["post_1"] + df["post_2"] + df["post_3"] + df["post_4"] + df["post_5"])*100

In [30]:
#Dummy for good/bad news (good=0, bad=1)

df["dummynews_goodbad"] = np.nan
df.loc[(df['pos_comparisons'] == 2) | (df['pos_comparisons'] == 3), 'dummynews_goodbad'] = 0
df.loc[(df['pos_comparisons'] == 0) | (df['pos_comparisons'] == 1), 'dummynews_goodbad'] = 1
df

,treatment,rscore,performance1,performance2,performance3,performance4,performance5,performance6,performance7,performance8,performance9,wspt,donation,prior_median,prior_1,prior_2,prior_3,prior_4,prior_5,prior_6,prior_7,prior_8,prior_9,prior_10,rank,pos_comparisons,neg_comparisons,repeat_neg_comparisons,repeat_pos_comparisons,risk_investment,recall_neg_comparisons,recall_pos_comparisons,big5_1,big5_2,big5_3,big5_4,big5_5,big5_6,big5_7,big5_8,big5_9,big5_10,big5_11,big5_12,big5_13,big5_14,big5_15,gender,field_of_study,student,posterior_median,recallfeedback,qualtrics,recall_part_IQtest,recall_part_feedback,recall_parts_nonIQ,pricelist1,pricelist2,pricelist3,pricelist4,pricelist5,pricelist6,pricelist7,pricelist8,pricelist9,pricelist10,pricelist11,pricelist12,pricelist13,pricelist14,pricelist15,treatgroup,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10,posterior_median_bayes,dummynews_goodbad
0,belief_announcement,4,1,3,4,4,5,5,5,6,8,6,1,53,2,4,9,14,24,16,13,9,7,2,7,2.0,1.0,1.0,2.0,80,1.0,2.0,1,2,4,3,2,4,3,4,2,5,3,4,3,4,4,0,Geographie,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0348774,0.1201635,0.2059946,0.3269755,0.1743869,0.0956403,0.0343324,0.0076294,0.0000000,68.8010899,0.0
1,belief_announcement,6,3,4,5,5,6,6,7,8,9,7,5,70,10,10,10,20,20,10,5,5,5,5,4,2.0,1.0,1.0,2.0,150,1.0,2.0,2,2,6,2,5,2,7,2,2,7,3,6,2,2,3,1,psychologie,1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0910384,0.1394026,0.3072546,0.2844950,0.1137980,0.0384068,0.0199147,0.0056899,0.0000000,82.2190612,0.0
2,belief_announcement,8,4,6,7,7,7,8,8,9,10,5,3,85,1,4,30,30,20,9,2,2,1,1,3,3.0,0.0,0.0,2.0,80,0.0,2.0,3,5,6,3,2,3,4,4,2,2,3,2,2,3,5,1,Medienwissenschaft,1,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.032123,0.0902441,0.4534238,0.2855380,0.1101613,0.0253812,0.0023795,0.0007050,0.0000441,0.0000000,97.1490262,0.0
3,belief_announcement,5,4,6,7,7,7,8,8,9,10,5,0,80,3,6,6,25,40,5,5,4,3,3,9,0.0,3.0,3.0,0.0,150,3.0,0.0,1,2,7,3,2,2,6,4,2,4,1,4,1,2,3,0,Lebensmittelchemie,1,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0005947,0.0047577,0.0669045,0.2537417,0.0619487,0.1070473,0.1359897,0.1522450,0.2167707,32.5998612,1.0
4,belief_announcement,5,1,3,4,4,5,5,5,6,8,3,0,25,0,0,0,0,25,5,5,20,20,25,4,3.0,0.0,0.0,3.0,100,0.0,3.0,4,4,7,3,2,1,7,2,6,3,2,4,1,1,4,1,Geographie,1,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0000000,0.0000000,0.0000000,0.8311170,0.0851064,0.0359043,0.0425532,0.0053191,0.0000000,83.1117021,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,tournament_announcement,7,3,4,5,5,6,6,7,8,9,5,0,80,5,10,40,15,10,4,4,4,4,4,3,2.0,1.0,1.0,2.0,0,1.0,2.0,2,2,6,2,5,4,6,2,4,5,3,5,2,3,3,0,Wirtschaftsinformatik,1,60.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0814249,0.4987277,0.2061069,0.1272265,0.0407125,0.0274809,0.0142494,0.0040712,0.0000000,91.3486005,0.0
738,tournament_announcement,7,4,6,7,7,7,8,8,9,10,4,4,75,10,10,30,15,10,10,5,5,5,0,8,0.0,3.0,3.0,0.0,200,3.0,0.0,1,1,2,2,2,3,6,1,1,3,1,7,2,1,3,1,Lehramt Englisch/Biologie Gym/Ge,1,50.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8,0.000000,0.0012658,0.0303797,0.0512658,0.0810127,0.1582278,0.1367089,0.2170886,0.3240506,0.0000000,16.3924051,1.0
739,tournament_announcement,7,4,6,7,7,7,8,8,9,10,5,0,70,2,8,20,30,10,12,8,6,3,1,6,0.0,3.0,3.0,0.0,0,3.0,0.0,3,2,5,3,4,3,5,4,3,3,3,2,3,4,4,0,Volkswirtschaftslehre,1,50.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.

 <span style="color:orange">**CONFIDENCE COMPARISON**

In [31]:
#Generate dummy for direct treatment and 1 month treatment
df["dummytreat_direct1month"] = np.nan
df.loc[(df['treatgroup'] == 3) | (df['treatgroup'] == 4), 'dummytreat_direct1month'] = 0
df.loc[df['treatgroup'] == 2, 'dummytreat_direct1month'] = 1
df

,treatment,rscore,performance1,performance2,performance3,performance4,performance5,performance6,performance7,performance8,performance9,wspt,donation,prior_median,prior_1,prior_2,prior_3,prior_4,prior_5,prior_6,prior_7,prior_8,prior_9,prior_10,rank,pos_comparisons,neg_comparisons,repeat_neg_comparisons,repeat_pos_comparisons,risk_investment,recall_neg_comparisons,recall_pos_comparisons,big5_1,big5_2,big5_3,big5_4,big5_5,big5_6,big5_7,big5_8,big5_9,big5_10,big5_11,big5_12,big5_13,big5_14,big5_15,gender,field_of_study,student,posterior_median,recallfeedback,qualtrics,recall_part_IQtest,recall_part_feedback,recall_parts_nonIQ,pricelist1,pricelist2,pricelist3,pricelist4,pricelist5,pricelist6,pricelist7,pricelist8,pricelist9,pricelist10,pricelist11,pricelist12,pricelist13,pricelist14,pricelist15,treatgroup,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10,posterior_median_bayes,dummynews_goodbad,dummytreat_direct1month
0,belief_announcement,4,1,3,4,4,5,5,5,6,8,6,1,53,2,4,9,14,24,16,13,9,7,2,7,2.0,1.0,1.0,2.0,80,1.0,2.0,1,2,4,3,2,4,3,4,2,5,3,4,3,4,4,0,Geographie,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0348774,0.1201635,0.2059946,0.3269755,0.1743869,0.0956403,0.0343324,0.0076294,0.0000000,68.8010899,0.0,NaN
1,belief_announcement,6,3,4,5,5,6,6,7,8,9,7,5,70,10,10,10,20,20,10,5,5,5,5,4,2.0,1.0,1.0,2.0,150,1.0,2.0,2,2,6,2,5,2,7,2,2,7,3,6,2,2,3,1,psychologie,1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0910384,0.1394026,0.3072546,0.2844950,0.1137980,0.0384068,0.0199147,0.0056899,0.0000000,82.2190612,0.0,NaN
2,belief_announcement,8,4,6,7,7,7,8,8,9,10,5,3,85,1,4,30,30,20,9,2,2,1,1,3,3.0,0.0,0.0,2.0,80,0.0,2.0,3,5,6,3,2,3,4,4,2,2,3,2,2,3,5,1,Medienwissenschaft,1,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.032123,0.0902441,0.4534238,0.2855380,0.1101613,0.0253812,0.0023795,0.0007050,0.0000441,0.0000000,97.1490262,0.0,NaN
3,belief_announcement,5,4,6,7,7,7,8,8,9,10,5,0,80,3,6,6,25,40,5,5,4,3,3,9,0.0,3.0,3.0,0.0,150,3.0,0.0,1,2,7,3,2,2,6,4,2,4,1,4,1,2,3,0,Lebensmittelchemie,1,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0005947,0.0047577,0.0669045,0.2537417,0.0619487,0.1070473,0.1359897,0.1522450,0.2167707,32.5998612,1.0,NaN
4,belief_announcement,5,1,3,4,4,5,5,5,6,8,3,0,25,0,0,0,0,25,5,5,20,20,25,4,3.0,0.0,0.0,3.0,100,0.0,3.0,4,4,7,3,2,1,7,2,6,3,2,4,1,1,4,1,Geographie,1,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0000000,0.0000000,0.0000000,0.8311170,0.0851064,0.0359043,0.0425532,0.0053191,0.0000000,83.1117021,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,tournament_announcement,7,3,4,5,5,6,6,7,8,9,5,0,80,5,10,40,15,10,4,4,4,4,4,3,2.0,1.0,1.0,2.0,0,1.0,2.0,2,2,6,2,5,4,6,2,4,5,3,5,2,3,3,0,Wirtschaftsinformatik,1,60.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0814249,0.4987277,0.2061069,0.1272265,0.0407125,0.0274809,0.0142494,0.0040712,0.0000000,91.3486005,0.0,NaN
738,tournament_announcement,7,4,6,7,7,7,8,8,9,10,4,4,75,10,10,30,15,10,10,5,5,5,0,8,0.0,3.0,3.0,0.0,200,3.0,0.0,1,1,2,2,2,3,6,1,1,3,1,7,2,1,3,1,Lehramt Englisch/Biologie Gym/Ge,1,50.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8,0.000000,0.0012658,0.0303797,0.0512658,0.0810127,0.1582278,0.1367089,0.2170886,0.3240506,0.0000000,16.3924051,1.0,NaN
739,tournament_announcement,7,4,6,7,7,7,8,8,9,10,5,0,70,2,8,20,30,10,12,8,6,3,1,6,0.0,3.0,3.0,0.0,0,3.0,0.0,3,2,5,3,4,3,5,4,3,3,3,2,3,4,4,0,Volkswirtschaftslehre,1,50.0,NaN,Na

In [32]:
#Generate interact
df["interact_direct1month"] = df["dummytreat_direct1month"]*df["dummynews_goodbad"]

In [33]:
#Reg1 for confidence
news_good = df[df["dummynews_goodbad"] == 0]
news_good = pd.DataFrame({"Y": news_good['posterior_median'], "D": news_good['dummytreat_direct1month']})
model_ols = smf.ols(formula="Y ~ D", data=news_good)
rslt = model_ols.fit(cov_type='HC1')
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                  0.001298
Date:                Fri, 25 Jun 2021   Prob (F-statistic):              0.971
Time:                        19:02:28   Log-Likelihood:                -605.59
No. Observations:                 138   AIC:                             1215.
Df Residuals:                     136   BIC:                             1221.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     79.2588      2.040     38.860      0.000      75.261      83.256
D             -0.1267      3.518     -0.036      0.971      -7.022       6.769
==============================================================================
Omnibus:                       26.837   Durbin-Watson:                   2.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.431
Skew:                          -1.044   Prob(JB):                     7.45e-09
Kurtosis:                       4.466   Cond. No.                         2.44
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [34]:
#Reg 2 for confidence
news_good = df[df["dummynews_goodbad"] == 0]
news_good = pd.DataFrame({"Y": news_good['posterior_median'], "D": news_good['dummytreat_direct1month'], "S": news_good["rank"]})
model_ols = smf.ols(formula="Y ~ D + S", data=news_good)
rslt = model_ols.fit(cov_type='HC1')
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.7580
Date:                Fri, 25 Jun 2021   Prob (F-statistic):              0.471
Time:                        19:02:28   Log-Likelihood:                -604.82
No. Observations:                 138   AIC:                             1216.
Df Residuals:                     135   BIC:                             1224.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     83.2147      3.532     23.560      0.000      76.292      90.137
D             -0.6824      3.543     -0.193      0.847      -7.627       6.262
S             -1.0988      0.894     -1.230      0.219      -2.850       0.653
==============================================================================
Omnibus:                       25.634   Durbin-Watson:                   2.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.252
Skew:                          -1.008   Prob(JB):                     2.21e-08
Kurtosis:                       4.438   Cond. No.                         10.7
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [35]:
#Reg 3 for confidence
news_bad = df[df["dummynews_goodbad"] == 1]
news_bad = pd.DataFrame({"Y": news_bad['posterior_median'], "D": news_bad['dummytreat_direct1month']})
model_ols = smf.ols(formula="Y ~ D", data=news_bad)
rslt = model_ols.fit(cov_type='HC1')
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     5.874
Date:                Fri, 25 Jun 2021   Prob (F-statistic):             0.0166
Time:                        19:02:28   Log-Likelihood:                -657.18
No. Observations:                 148   AIC:                             1318.
Df Residuals:                     146   BIC:                             1324.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     45.9462      2.295     20.018      0.000      41.448      50.445
D              8.0719      3.331      2.424      0.015       1.544      14.600
==============================================================================
Omnibus:                        1.520   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.468   Jarque-Bera (JB):                1.097
Skew:                           0.169   Prob(JB):                        0.578
Kurtosis:                       3.252   Cond. No.                         2.43
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [36]:
#Reg 4 for confidence
news_bad = df[df["dummynews_goodbad"] == 1]
news_bad = pd.DataFrame({"Y": news_bad['posterior_median'], "D": news_bad['dummytreat_direct1month'], "S": news_bad["rank"]})
model_ols = smf.ols(formula="Y ~ D + S", data=news_bad)
rslt = model_ols.fit(cov_type='HC1')
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     8.146
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           0.000444
Time:                        19:02:28   Log-Likelihood:                -652.57
No. Observations:                 148   AIC:                             1311.
Df Residuals:                     145   BIC:                             1320.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     66.3473      6.982      9.502      0.000      52.662      80.032
D              7.2644      3.278      2.216      0.027       0.840      13.689
S             -2.6874      0.885     -3.035      0.002      -4.423      -0.952
==============================================================================
Omnibus:                        3.051   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.218   Jarque-Bera (JB):                2.543
Skew:                           0.274   Prob(JB):                        0.280
Kurtosis:                       3.334   Cond. No.                         33.5
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [37]:
#Reg 5 for confidence
df_c5 = pd.DataFrame({"Y": df['posterior_median'], "X_1": df['dummytreat_direct1month'], "X_2": df["dummynews_goodbad"], "X_3": df["interact_direct1month"]})
model_ols = smf.ols(formula="Y ~ X_1 + X_2 + X_3", data=df_c5)
rslt = model_ols.fit(cov_type='HC1')
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.371
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     54.21
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           1.07e-27
Time:                        19:02:28   Log-Likelihood:                -1263.0
No. Observations:                 286   AIC:                             2534.
Df Residuals:                     282   BIC:                             2549.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     79.2588      2.039     38.870      0.000      75.262      83.255
X_1           -0.1267      3.517     -0.036      0.971      -7.021       6.767
X_2          -33.3126      3.071    -10.849      0.000     -39.331     -27.294
X_3            8.1987      4.845      1.692      0.091      -1.296      17.694
==============================================================================
Omnibus:                       11.673   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               13.866
Skew:                          -0.370   Prob(JB):                     0.000975
Kurtosis:                       3.785   Cond. No.                         6.46
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [38]:
#Reg 6 for confidence
df_c6 = pd.DataFrame({"Y": df['posterior_median'], "X_1": df['dummytreat_direct1month'], "X_2": df["dummynews_goodbad"], "X_3": df["interact_direct1month"], "X_4":df["rank"]})
model_ols = smf.ols(formula="Y ~ X_1 + X_2 + X_3 + X_4", data=df_c6)
rslt = model_ols.fit(cov_type='HC1')
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     44.82
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           4.31e-29
Time:                        19:03:10   Log-Likelihood:                -1258.3
No. Observations:                 286   AIC:                             2527.
Df Residuals:                     281   BIC:                             2545.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     86.1788      2.875     29.971      0.000      80.543      91.814
X_1           -1.0987      3.548     -0.310      0.757      -8.053       5.855
X_2          -25.6402      3.970     -6.458      0.000     -33.422     -17.859
X_3            8.5931      4.801      1.790      0.073      -0.816      18.002
X_4           -1.9222      0.626     -3.069      0.002      -3.150      -0.695
==============================================================================
Omnibus:                       10.031   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               12.313
Skew:                          -0.311   Prob(JB):                      0.00212
Kurtosis:                       3.804   Cond. No.                         32.5
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

<span style="color:orange">**BELIEF ADJUSTMENT**

In [39]:
##Bayesian prediction for belief adjustment
df["beliefadjustment_bayes"] = df["posterior_median_bayes"] - df["prior_median"]
#norm
df["beliefadjustment_bayes_norm"] = np.nan
df.loc[df['dummynews_goodbad'] == 0, 'beliefadjustment_bayes_norm'] = df['beliefadjustment_bayes']
df.loc[df['dummynews_goodbad'] == 1, 'beliefadjustment_bayes_norm'] = df['beliefadjustment_bayes']*-1

In [40]:
#normalized belief adjustment
df["beliefadjustment"] = df["posterior_median"] - df["prior_median"]
#normalized belief adjustment_normalized
df["beliefadjustment_normalized"] = np.nan
df.loc[df['dummynews_goodbad'] == 0, 'beliefadjustment_normalized'] = df['beliefadjustment']
df.loc[df['dummynews_goodbad'] == 1, 'beliefadjustment_normalized'] = df['beliefadjustment']*-1

Direct vs 15 min

In [41]:
#Generate dummytreat_direct15min
df["dummytreat_direct15min"] = np.nan
df.loc[df['treatgroup'] == 4, 'dummytreat_direct15min'] = 0
df.loc[df['treatgroup'] == 3, 'dummytreat_direct15min'] = 1

In [42]:
#so only bad news and 15min later
df["interact_direct15min"] = df["dummytreat_direct15min"]*df["dummynews_goodbad"]

# 7. Conclusion
---

---
# 8. References
---